In [1]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
%reload_ext autoreload
%autoreload 2
from HelperFunctions import feature_standardize, label_encode_column, columns_of_type

from sklearn.model_selection import GridSearchCV
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score

from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
reg = RandomForestRegressor()
clf = RandomForestClassifier()

import pickle

In [2]:
temp_pre_df=pd.read_csv('pre_downsample_df.csv')
pre_df=temp_pre_df.sample(50000)
holdout=pd.read_csv('Holdout_Dataset.csv')

In [3]:
holdout.head()

,acc_now_delinq,addr_state,all_util,annual_inc,application_type,chargeoff_within_12_mths,collections_12_mths_ex_med,delinq_2yrs,delinq_amnt,dti,emp_length,funded_amnt,grade,home_ownership,il_util,initial_list_status,inq_last_12m,int_rate,issue_d,mo_sin_rcnt_tl,mths_since_last_delinq,mths_since_last_major_derog,mths_since_last_record,num_accts_ever_120_pd,num_rev_accts,num_tl_90g_dpd_24m,num_tl_op_past_12m,open_acc,num_il_tl,pct_tl_nvr_dlq,pub_rec,pub_rec_bankruptcies,purpose,revol_util,sub_grade,tax_liens,term,tot_coll_amt,tot_cur_bal,tot_hi_cred_lim,total_acc,total_bc_limit,total_il_high_credit_limit,total_rev_hi_lim,verification_status,loan_status,fico_score,active_rev_acct_frac,active_ins_acct_frac,total_payout,duration,return_rate,credit_hist,treasury_rate,unemp_rate_3mon
0,0.0,NY,54.0,65000.0,Individual,0.0,0.0,0.0,0.0,24.89,5,15000.0,B,RENT,65.0,f,2.0,11.39,2017-03-01,10.0,-999.0,-999.0,-999.0,0.0,12.0,0.0,1.0,10.0,4.0,100.0,0.0,0.0,other,45.1,B3,0.0,36,476.0,37371.0,70746.0,16.0,16300.0,50946.0,19800.0,Source Verified,Fully Paid,697.0,0.333333,0.5,15901.83,0.587443,0.104493,9.756164,1.59,5.10
1,0.0,TX,-999.0,50000.0,Individual,0.0,0.0,0.0,0.0,27.29,1,15000.0,C,OWN,-999.0,w,-999.0,13.33,2015-04-01,10.0,63.0,-999.0,-999.0,0.0,13.0,0.0,3.0,9.0,2.0,94.7,0.0,0.0,debt_consolidation,80.2,C3,0.0,36,0.0,44783.0,70377.0,19.0,32000.0,38377.0,32000.0,Source Verified,Fully Paid,702.0,0.538462,-999.0,18186.92,2.924429,0.068095,33.021918,0.87,4.23
2,0.0,VA,-999.0,70000.0,Individual,0.0,0.0,0.0,0.0,17.28,10,11700.0,B,MORTGAGE,-999.0,f,-999.0,9.49,2015-01-01,3.0,66.0,-999.0,-999.0,0.0,29.0,0.0,3.0,20.0,4.0,97.4,0.0,0.0,debt_consolidation,29.4,B2,0.0,60,0.0,201201.0,269376.0,38.0,31000.0,27876.0,57000.0,Source Verified,Fully Paid,737.0,0.241379,-999.0,13173.93,1.666895,0.073775,23.183562,1.37,4.87
3,0.0,GA,-999.0,80000.0,Individual,0.0,0.0,0.0,0.0,29.06,5,26400.0,G,RENT,-999.0,w,-999.0,25.83,2013-08-01,3.0,-999.0,-999.0,-999.0,0.0,13.0,0.0,4.0,14.0,15.0,100.0,0.0,0.0,debt_consolidation,83.7,G2,0.0,60,0.0,84676.0,102699.0,28.0,29500.0,69499.0,33200.0,Verified,Fully Paid,682.0,0.461538,-999.0,47165.73,4.751826,0.129892,12.008219,1.52,8.63
4,0.0,NY,29.0,63000.0,Individual,0.0,0.0,0.0,0.0,9.47,2,4800.0,A,RENT,71.0,f,0.0,6.19,2018-06-01,0.0,-999.0,-999.0,-999.0,0.0,12.0,0.0,2.0,14.0,12.0,100.0,0.0,0.0,debt_consolidation,1.8,A2,0.0,36,0.0,19481.0,67055.0,24.0,34900.0,26455.0,40600.0,Not Verified,Fully Paid,792.0,0.083333,0.5,4924.90,0.584703,0.044913,13.175342,2.65,3.93


In [4]:
drop_list=['sub_grade','issue_d','duration','loan_status','return_rate','total_payout']
holdout.drop(drop_list,axis=1,inplace=True)
cat_list=columns_of_type(holdout,'string')
holdout_encoded=label_encode_column(holdout,cat_list)
# holdout.groupby('grade', group_keys=False).apply(lambda x: x.sample(min(len(x), 2)))

### adjustment of probability from classification model (to address downsampling)

In [5]:
# def grade_default_rate(unsampled_df): 
#     # Finding the Default rate of each grade (of a given time)
#     grade_rate=unsampled_df.groupby('grade')['loan_status'].apply(lambda x:(x=='Default').sum()/x.count())
#     grade_rate_dict=grade_rate.to_dict()
#     return grade_rate_dict
       
def adjust_prob(grade_rate_dict,df,pre_adjust_prob):
    pre_adjust_prob_df=pd.DataFrame(pre_adjust_prob,columns=['pre_adjust_prob'])
    df['grade_default_rate']=df['grade'].map(grade_rate_dict)
    df.reset_index(drop=True, inplace=True)
    pre_adjust_prob_df.reset_index(drop=True, inplace=True)
    pre_adjust_df=pd.concat([df,pre_adjust_prob_df],axis=1)
    # Adjusting the default_probability to the true probability (accounting for down/up sampling)    
    sampled_frac=0.5
    real_prob=[]
    for row in pre_adjust_df.loc[:,['grade_default_rate','pre_adjust_prob']].iterrows():
        beta=sampled_frac/(1-row[1]['grade_default_rate'])
        real_prob.append(beta*row[1]['pre_adjust_prob']/((beta-1)*row[1]['pre_adjust_prob']+1))
    a=pd.DataFrame(real_prob,columns=['actual_prob'])
    b=pd.DataFrame(pre_adjust_prob,columns=['pre_adjust_prob'])
    return pd.concat([a,b],axis=1)

In [6]:
def grade_weighted_accuracy(y_true,y_pred,sample_weight):
        weighted_acc=accuracy_score(y_true,y_pred,
                                    sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1),
                                    normalize=True)
        return weighted_acc 

In [7]:
def grade_weighted_r2(y_true,y_pred,sample_weight):        

            weighted_r2=r2_score(y_true,y_pred,sample_weight=sample_weight.loc[y_true.index.values].values.reshape(-1))
            return weighted_r2    

# Load Pickle Files

In [8]:
numberTimeWindow=22

In [9]:
pickle_model_clf=[]
with open('pickle_model_clf.pkl', 'rb') as file:
    for time in range(0,numberTimeWindow):
        pickle_model_clf.append(pickle.load(file))

In [10]:
pickle_model_reg_def=[]
with open('pickle_model_reg_def.pkl', 'rb') as file:
    for time in range(0,numberTimeWindow):
        pickle_model_reg_def.append(pickle.load(file))

In [11]:
pickle_model_reg_paid=[]
with open('pickle_model_reg_paid.pkl', 'rb') as file:
    for time in range(0,numberTimeWindow):
        pickle_model_reg_paid.append(pickle.load(file))

In [12]:
pickle_time_grade_default_rate=[]
with open('pickle_time_grade_default_rate.pkl', 'rb') as file:
    for time in range(0,numberTimeWindow):
        pickle_time_grade_default_rate.append(pickle.load(file))

In [13]:
finaldf=[]
expected_return_rate=[]
expected_prob=[]

In [27]:
pickle_time_grade_default_rate[0].values()

dict_values([0.05555555555555555, 0.3, 0.2702702702702703, 0.34375, 0.3181818181818182, 0.2857142857142857, 0.3333333333333333])

# Calculating the expected return rate for each time frame

In [14]:
for time in range(0,numberTimeWindow):
    
#     time=1
    print(time)
    grade_rate_dict=pickle_time_grade_default_rate[time]
    pre_adjust_prob=pickle_model_clf[time].predict_proba(holdout_encoded)
    pre_adjust_predict=pickle_model_clf[time].predict(holdout_encoded)

    # Getting the right prbability for default probability rate:
    if (pre_adjust_predict[0]=='Fully Paid') and (pre_adjust_prob[0,0]<0.5):
        correct_label_pre_adjust_prob=pre_adjust_prob[:,1]
    elif (pre_adjust_predict[0]=='Fully Paid') and (pre_adjust_prob[0,0]>0.5):
        correct_label_pre_adjust_prob=pre_adjust_prob[:,0]
    elif (pre_adjust_predict[0]=='Default') and (pre_adjust_prob[0,0]>0.5):
        correct_label_pre_adjust_prob=pre_adjust_prob[:,0]
    elif (pre_adjust_predict[0]=='Default') and (pre_adjust_prob[0,0]<0.5):
        correct_label_pre_adjust_prob=pre_adjust_prob[:,1]
    else: 
        print(pre_adjust_predict[0])
        raise TypeError('loan status not right')

    prob_df=adjust_prob(pickle_time_grade_default_rate[time],holdout,correct_label_pre_adjust_prob)

    def_rate=pickle_model_reg_def[time].predict(holdout_encoded)
    paid_rate=pickle_model_reg_paid[time].predict(holdout_encoded)
    
    expected_return_rate.append(def_rate*prob_df['actual_prob']+paid_rate*(1-prob_df['actual_prob']))
    expected_prob.append(prob_df)
    

0
1
2
3


/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:17: RuntimeWarning: divide by zero encountered in double_scalars
/home/auscheng/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


# Ensembling all the return rates of all the time frames

In [15]:
def weighted_sum(f1,f2):
    return(f1*0.4+f2*0.6)

In [17]:
# total_return=expected_return_rate[0]
total_prob=expected_prob[0]
# for ind in range(0,len(expected_return_rate)-1):
#     total_return=weighted_sum(total_return,expected_return_rate[ind+1])    
for ind in range(0,len(expected_prob)-1):
    total_prob=weighted_sum(total_prob,expected_prob[ind+1]) 

In [20]:
total_prob.min()

actual_prob        0.182305
pre_adjust_prob    0.298930
dtype: float64

# Getting the default probability rate of each observation

In [ ]:
final_df=pd.DataFrame([total_return,total_prob])

In [ ]:
final_df.head()